## Trực quan hóa Word2Vec
/> Sử dụng tensorboard để trực quan hóa word2vec.

In [2]:
import warnings
warnings.filterwarnings(action='ignore')


import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import gensim
import os

/> Load model word2vec.

In [4]:
file_name = "./model/word2vec.model"
model = gensim.models.keyedvectors.KeyedVectors.load(file_name)

/> Sau khi load model thành công, chúng ta cần lưu lại số từ vựng có trong model.

In [6]:
max_size = len(model.wv.key_to_index.keys())-1

/> Chúng ta biết rằng số chiều của các word vector là $\boldsymbol{V} \times \boldsymbol{N}$. Tức số từng vựng $\boldsymbol{V} \times$ số neuron trong hidden layer $\boldsymbol{N}$. Bây giờ chúng ta sẽ tạo ma trận $\boldsymbol{W}$ và lưu vào biến `w2v`.

In [7]:
w2v = np.zeros((max_size, model.layer1_size))

/> Tiếp theo, chúng ta sẽ lưu các từ cùng các word vectors vào file `metadata.tsv`.

In [9]:
if not os.path.exists('projections'):
    os.makedirs('projections')
    
with open("projections/metadata.tsv", 'w+') as file_metadata:    
    for i, word in enumerate(model.wv.index_to_key[:max_size]):
        # lưu embedding vector
        w2v[i] = model.wv[word]
        
        # lưu word tương ứng 
        file_metadata.write(word + '\n')

/> Khởi tạo TensorFlow session.

In [11]:
sess = tf.InteractiveSession()

2022-02-08 12:15:47.301965: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 12:15:47.302125: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce GTX 1060 major: 6 minor: 1 memoryClockRate(GHz): 1.6705
pciBusID: 0000:01:00.0
2022-02-08 12:15:47.302176: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2022-02-08 12:15:47.302191: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-02-08 12:15:47.302202: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2022-02-08 12:15:47.302214: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic libra

/> Tạo biến `embeddings` để chứa các từ nhúng và các thiết lập cần thiết cho Tensorboard.

In [12]:
with tf.device("/gpu:0"):
    embedding = tf.Variable(w2v, trainable=False, name='embedding')

In [13]:
tf.global_variables_initializer().run()

2022-02-08 12:21:42.008374: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 22456000 exceeds 10% of system memory.
2022-02-08 12:21:42.018583: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 22456000 exceeds 10% of system memory.


In [14]:
saver = tf.train.Saver()

In [15]:
writer = tf.summary.FileWriter('projections', sess.graph)

In [16]:
config = projector.ProjectorConfig()
embed = config.embeddings.add()

/> Chỉ định tập tin để `metadata.tsv` để trực quan hóa bằng Tensorboard.

In [17]:
embed.tensor_name = 'embedding'
embed.metadata_path = 'metadata.tsv'

/> Lưu lại model dùng để visualization của TensorBoard.

In [18]:
projector.visualize_embeddings(writer, config)
saver.save(sess, 'projections/model.ckpt', global_step=max_size)

'projections/model.ckpt-28070'

/> Mở terminal và nhập lệnh dưới đây để trực quan hóa.
  ```shell
  tensorboard --logdir=projections --port=8000
  ```

  ![](./images/01.png)